# Battle of the Neighbourhoods
This capstone assignment will investigate the characteristics of different neighbourhoods in Sydney Australia to help our family relocate. In the assignemnt we use publicly available data to examine rent prices, school options, crime statistics and character of different neighbourhoods. 

We cluster neighbourhoods using a k-means algorithm to group similar suburbs and identify those that offer family friendly activities, good schools at affordable prices

# The Datasets

## 1. Schools Data
Schools data and School location data is obtained from the Australian Curriculum Assessment and Reporting Austhority (ACARA). The advantage of this dataset is that it includes both public and private school sectors. However school asessment data of NAPLAN results is not avialable to private individuals although data could be scraped from a site one school at a time. 

School Profile Data:
https://www.acara.edu.au/docs/default-source/default-document-library/school-profile-2018.xlsx?sfvrsn=0 
School Location Data: 
https://www.acara.edu.au/docs/default-source/default-document-library/school-locations-20189cf512404c94637ead88ff00003e0139.xlsx?sfvrsn=0

Each school is identified by a unique ACARA id so school profile and location data can be easily joined. The school location is geocoded and can easily be displayed on a map.

## 2. Postcode Geocoded Data
Geocoded postcode data is required to support the rental and crime datasets whose granularity is logged at postcode level. Note in Australia a single postcode may span multiple suburbs. Unfortunately geocoded postcode data is no freely available since the postal service decided to monetise the data. There are a number of internet sources available that vary in terms of comprehensiveness and currency. This project has selected the data provided by [Matthew proctor](https://www.matthewproctor.com/australian_postcodes) as the most comprehensive and recent source of geolocated postcode data.

As postcodes areas do not necessarily correspond directly to electroral or parish boundaries the most accurate way to visualise data encoded via postcode is to use a geojson file of postcode boundaries. This can then be used in conjuction with Folium to create a postal area overlay that will map to the granularity of of data. There is no official geojson file although the Australian Bureau of statistics publishes mapinfo and shape files of [postcode boundaries](https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.003July%202011?OpenDocument).

Fortunately the work in transforming the official postcode shape files into a usable postcode area geojson file was performed by Luke Singham and may be downloaded from his github repo [here](https://github.com/ucg8j/aus_cloropleth/blob/master/au-postcodes-Visvalingam-0.1-density.js). It is possible to generate the geojson file directly from the shap file using a utility called mapshaper which mat be downloaded using the node package manager. However as the postcode shape file has not changed since 2011 the visualisations are based on Luke Signham's geojson data.

## 4. Crime Data
Crime data is obtained from the New South Wales Bureau of Crime Statistics and Research (BOSCAR)



## 3. Neighbourhood Data
Obtained using the foursquare API



In [1]:
# Install Libraries
!conda install -c conda-forge folium --yes
!conda install -c conda-forge wget --yes

Solving environment: \ 
  - anaconda/osx-64::ca-certificates-2019.8.28-0, anaconda/osx-64::certifi-2019.9.11-py37_0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_2
  - anaconda/osx-64::ca-certificates-2019.8.28-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_2, defaults/osx-64::certifi-2019.9.11-py37_0
  - anaconda/osx-64::certifi-2019.9.11-py37_0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_2, defaults/osx-64::ca-certificates-2019.8.28-0
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_2, defaults/osx-64::ca-certificates-2019.8.28-0, defaults/osx-64::certifi-2019.9.11-py37_0
  - anaconda/osx-64::certifi-2019.9.11-py37_0, defaults/osx-64::ca-certificates-2019.8.28-0, defaults/osx-64::openssl-1.1.1d-h1de35cc_2
  - defaults/osx-64::ca-certificates-2019.8.28-0, defaults/osx-64::certifi-2019.9.11-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_2
  - anaconda/osx-64::ca-certificates-2019.8.28-0, anaconda/osx-64::certifi-2019.9.11-py37_0, defaults/osx-64::openssl-1.1.1d-h1de35cc_2
  - anaconda/osx-64::ca-

In [2]:
# Import Libraries
import pandas as pd
import requests
import io
import folium
from bs4 import BeautifulSoup

# Load Postcode Data File

Postcode data file is avilable from [here](https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv). The fields in this file are defined as:

## Geocoded Postcode Dataset Fields

| Field Name | Description |
| --- | --- |
| id         | Primary Key from source Database | 
| postcode   | Postcode in numeric format 0000 |
| locality   | Area or neighbourhood covered by the postcode |
| state      | State the postcode belongs to. Note different state may use the same postcode |
| long       | longitude |
| lat        | latitude |
| dc         | postal distribution centre |
| type       | type of locality e.g. PO box, Large Volume Recipient (LVR) |
| status     | new, removed or updated |


In [3]:
# Use wget to download the required file
postcode_suburb_file_source = "https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv"
postcode_suburb_file_target = "./data/australian_postcodes_suburb_geo.csv"
!wget -O $postcode_suburb_file_target $postcode_suburb_file_source

--2020-02-03 18:01:55--  https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv
Resolving www.matthewproctor.com (www.matthewproctor.com)... 2606:4700:3032::681b:aa10, 2606:4700:3033::681b:ab10, 104.27.170.16, ...
Connecting to www.matthewproctor.com (www.matthewproctor.com)|2606:4700:3032::681b:aa10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1671663 (1.6M) [application/octet-stream]
Saving to: ‘./data/australian_postcodes_suburb_geo.csv’

./data/australian_p 100%[===================>]   1.59M  2.14MB/s    in 0.7s    

2020-02-03 18:01:56 (2.14 MB/s) - ‘./data/australian_postcodes_suburb_geo.csv’ saved [1671663/1671663]



In [4]:
# Sydney Inner Citry Post codes are all in the range 2000-2999
# Set up a list of postcodes to be used as a filter
sydney_post_codes = [str(item) for item in range(2000,3000)]
sydney_lat_lng = [-33.865143, 151.209900]


In [5]:
# Read the School Profile Data Into a dataframe
df_geo_postcodes_suburb = pd.read_csv(postcode_suburb_file_target)

# Drop the dc column as we are not interested in postal distribution centre
df_geo_postcodes_suburb.drop("dc", axis=1, inplace=True)

# Update the data types 
df_geo_postcodes_suburb["postcode"] = df_geo_postcodes_suburb["postcode"].astype(str)

# We are only interested in New South Wales so get rid of all other state data
df_geo_postcodes_suburb_nsw = df_geo_postcodes_suburb[df_geo_postcodes_suburb["state"]=="NSW"]
print(df_geo_postcodes_suburb_nsw.shape)

# We can also ignore postcodes that are Large Volume Receivers or Postal Box addresses
# According to Wikipedia these have a postcode in the range 1000-1999
# Valid NSW postcodes are all in the range 2000-2999
# See https://en.wikipedia.org/wiki/Postcodes_in_Australia for further information
df_geo_postcodes_suburb_nsw = df_geo_postcodes_suburb_nsw[df_geo_postcodes_suburb_nsw["postcode"].isin(sydney_post_codes)]
print(df_geo_postcodes_suburb_nsw.shape)
print(df_geo_postcodes_suburb_nsw["type"].value_counts())

# Value counst shows data is not entirely consistent with wikipedia characterisation and
# there are residual LVR and PO Box codes inside the 2000-2999 range. We'll remove these as well
#df_geo_postcodes_nsw = df_geo_postcodes_nsw[~df_geo_postcodes_nsw["type"].isin(["Post Office Boxes","LVR"])]
#print(df_geo_postcodes_nsw["type"].value_counts())

# See where we have null values in the dataset
df_geo_postcodes_suburb_nsw.isnull().sum()
df_geo_postcodes_suburb_nsw.head()

(5559, 7)
(5225, 7)
Delivery Area        4877
Post Office Boxes      11
LVR                     9
Name: type, dtype: int64


,id,postcode,locality,state,long,lat,type
567,20208,2000,BARANGAROO,NSW,151.201580,-33.860520,Delivery Area
568,4478,2000,DARLING HARBOUR,NSW,151.256649,-33.859953,NaN
569,4479,2000,DAWES POINT,NSW,151.256649,-33.859953,Delivery Area
570,4480,2000,HAYMARKET,NSW,151.256649,-33.859953,Delivery Area
571,4481,2000,MILLERS POINT,NSW,151.256649,-33.859953,Delivery Area


In [13]:
# Load geocoded postcode file. This defines the central geocoded lat lng for each postcode area, not broken down by suburb
postcode_file_target = "./data/australian_postcodes_geo.csv"
df_geo_postcodes = pd.read_csv(postcode_file_target, header=None, names=["postcode","area","state","lat","long"])
df_geo_postcodes = df_geo_postcodes[df_geo_postcodes["state"]=="NSW"]
df_geo_postcodes.head()

#Decimal Degrees = degrees + (minutes/60) + (seconds/3600)
-33 + 0.86666 + 0..0083333


,postcode,area,state,lat,long
26,1363,MOORE PARK,NSW,NaN,NaN
27,1639,FRENCHS FOREST,NSW,NaN,NaN
28,2000,THE ROCKS,NSW,-33:52:30,-151:12:5
29,2006,THE UNIVERSITY OF SYDNEY,NSW,-33:53:30,-151:11:06
30,2007,ULTIMO,NSW,-33:52:45,-151:11:49


# Load the Schools Datasets

Schools data and School location data is obtained from the Australian Curriculum Assessment and Reporting Austhority (ACARA)

## Schools Profile Dataset Fields

| Field Name | Description |
| --- | --- |
| Calendar Year      | School Calendar Year (this dtaset is for 2018, te latest available) |
| ACARA SML ID       | Acara issued unique school identifier |
| Age Id             | Australian Dovernment Department Education Id |
| School Name        | Name of the school |
| Suburb             | Location (Suburb Name) |
| State              | Location (State Name) |
| Postcode           | Location Postcode |
| School Sector      | Sector (Government, Private, Independent, Faith based) |
| School Type        | Primary, Secondary |
| Campus Type        | Single School or academy identifier |
| Rolled Reporting   | Field for reporting in ACARA portal |
| School URL         | School website URL |
| Governing Body     | Name of governing body  |
| Governing Body URL | Governing Body website URL |
| Year Range         | The range of year levels offered by the school |
| Geolocation        | Geographical classification Major Cities, Inner Regional, Outer Regional, Remote, Very Remote |
| ICSEA              | The Index of Community Socio‐Educational Advantage score for the school |
| Teaching Staff     | Number of staff |
| FTE Teaching Staff | Full time equivalent teaching staff  |
| Non Teaching Staff | Non tecahing staff |
| FTE Non Teaching   | Full time equivalent teaching staff |
| Total Enrollments  | Total pupils enrolled |
| Girls Enrolled     | Total girls enrolled  |
| Boys Enrolled      | Total boys enrolled |
| FTE enrollments    | FTE pupil enrolments |
| Indigenous Enrols  | % of indigenous pupils |
| Non English        | % of pupils with English as a second language |
| Bottom SEA Quarter (%)       | The % of students in the lowest socio‐educational advantage quarter |
| Lower Middle SEA Quarter (%) | The % of students in the lower-middle socio‐educational advantage quarter  |
| Upper Middle SEA Quarter (%) | The % of students in the upper-middle socio‐educational advantage quarter |
| Top SEA Quarter (%)          | The % of students in the highest socio‐educational advantage quarter |

## Schools Location Dataaet Fields

| Field Name | Description |
| --- | --- |
| Calendar Year      | School Calendar Year (this dtaset is for 2018, te latest available) |
| ACARA SML ID       | Acara issued unique school identifier |
| Age Id             | Australian Dovernment Department Education Id |
| School Name        | Name of the school |
| Suburb             | Location (Suburb Name) |
| State              | Location (State Name) |
| Postcode           | Location Postcode |
| School Sector      | Sector (Government, Private, Independent, Faith based) |
| School Type        | Primary, Secondary |
| Campus Type        | Single School or academy identifier |
| Latitude           | Location latitude |
| Longitude          | Location Longitude |
| Statistical Area 1 | Code for location (statistical grouping) |
| Statistical Area 2 | Code for location (statistical grouping) |
| Statistical Area 3 | Code for location (statistical grouping) |
| Statistical Area 4 | Code for location (statistical grouping) |
| Name Statistical Area 2 | Descriptive name for location (statistical grouping) |
| Name Statistical Area 3 | Descriptive name for location (statistical grouping) |
| Name Statistical Area 4 | Descriptive name for location (statistical grouping) |
| ABS Remoteness Indicatr | Remoteness description |


In [ ]:
# Download the Schools Profile Data
school_profile_source = "https://www.acara.edu.au/docs/default-source/default-document-library/school-profile-2018.xlsx?sfvrsn=0"
school_profile_target = "./data/acara-school-profile-2018.xlsx"
!wget -O $school_profile_target $school_profile_source

# Download the Schools Location Data
school_location_source = "https://www.acara.edu.au/docs/default-source/default-document-library/school-locations-20189cf512404c94637ead88ff00003e0139.xlsx?sfvrsn=0"
school_location_target = "./data/acara-school-locs-2018.xlsx"
!wget -O $school_location_target $school_location_source

In [ ]:
# Read the School Profile Data Into a dataframe
df_schools = pd.read_excel(school_profile_target, sheet_name=1)
df_schools.head()

# Read the school locations data into a dataframe
df_school_locs = pd.read_excel(school_location_target, sheet_name=1)
df_school_locs.head()

In [ ]:
# Merge the two datasets together by adding location data into the school profile data

# First Get the Columns in the locations datafrane that are not also repersented in the school profile datafrane
# Easiest way to do this is to push the columsn name data into sets and run a difference
# However we will need to add the common key back in to join the data
set_prf_cols = set(df_schools.columns)
set_loc_cols = set(df_school_locs.columns)
extract_cols = list(set_loc_cols.difference(set_prf_cols))
extract_cols.sort()
extract_cols.insert(0, "ACARA SML ID")

# Run the merge using the Acara SML ID as a common join key
df_schools_merged = pd.merge(df_schools, df_school_locs[extract_cols], left_on="ACARA SML ID", right_on="ACARA SML ID", how="left")
df_schools_merged.shape


In [ ]:
# Drop the columns that are not useful for our analysis
df_schools_merged.columns
columns_to_drop =["Rolled Reporting Description","School URL","Governing Body","Governing Body URL", "Statistical Area 1", "Statistical Area 2", "Statistical Area 3", "Statistical Area 4"]
df_schools_merged.drop(columns_to_drop, axis=1, inplace=True)
df_schools_merged.columns

In [ ]:
# Filter the merged dataset to focus only on schools in New South Wales
df_schools_nsw = df_schools_merged[df_schools_merged["State"]=="NSW"]

# Filter the merged dataset to focus only on schools that are girls only or mixed
df_schools_nsw=df_schools_nsw[df_schools_nsw["Girls Enrolments"] > 0]

# Filter the merged dataset to focus only on schools that primary or combined
df_schools_nsw=df_schools_nsw[df_schools_nsw["School Type"].isin(["Primary","Combined"])]

# Filter the merged dataset to focus only on schools that primary or combined
df_schools_syd_girls_prim=df_schools_nsw[df_schools_nsw["Name of Statistical Area 4"].str.contains("Sydney")]

# Filter the merged dataset to focus only on schools that are within Sydney region
df_schools_syd_girls_prim.head() 



In [ ]:
# Summarise the number of girls primamry schools in each area of Sydney
df_schools_syd_girls_prim["Name of Statistical Area 4"].value_counts()

# Get the Local Crime Data

This data is sourced from the Bureau of Crime Statistics and Research (BOSCAR). We have cosen a dataset that is keyed on post code to enable consistent joining of the data to other data sources we have obtained. For this exercise we will use the Recoreded Crime by Offence dataset broken down to postcode granuality. The dataset is published [here](https://www.bocsar.nsw.gov.au/Pages/bocsar_datasets/Datasets-.aspx) and covers the period until 2018. 

## Recorded Crimes By Postcode Dataset

| Field | Description |
| --- | --- |
| Postcode         | Postcode |
| Offence category | Categorisation |
| Subcategory      | Sub Categorisation |
| Jan 95 | Start Month and Year in reported data (Jan 1995 - Dec 2018) |
| ... | ... |
| Dec 18 | End Month and Year in reported data (Jan 1995 - Dec 2018) |


In [ ]:
# Download the Local Crime Statistics Data
crime_data_source = "https://www.bocsar.nsw.gov.au/Documents/Datasets/PostcodeData.zip"
crime_data_zipped = "./data/crime_by_postcode2018.zip"
crime_data_unzipd = "./data/PostcodeData2018.csv"
crime_data_folder = "./data/"
crime_data_target = "./data/crime_by_postcode2018.csv"

!wget -O $crime_data_zipped $crime_data_source

# Unzip the crime data file
!unzip $crime_data_zipped -d $crime_data_folder
print("File Successfully Unzipped")

!mv -f $crime_data_unzipd $crime_data_target
print("File Successfully Renamed")

In [ ]:
# Read the Data into a dataframe
df_crimes = pd.read_csv(crime_data_target)
df_crimes.shape
df_crimes.head()

In [ ]:
# lets examine the data types loaded and change the postcode to a string
df_crimes["Postcode"] = df_crimes["Postcode"].astype(str)
df_crimes.dtypes

In [ ]:
# Aggregate the monthy data into years as we don't need to know crime data to monthly granularty.
# This will also make the data more manageable
df_crimes_agg = df_crimes[["Postcode","Offence category","Subcategory"]].copy()

for item_year in range (1995,2019) :
    current_year = str(item_year)
    df_crimes_agg[current_year] = df_crimes.filter(regex="{}".format(current_year)).sum(axis=1)

df_crimes_agg.shape

In [ ]:
# Remove Data out of the sydney postcose range 2000-2999 as we are not interested in outliying rurual areas
df_crimes_agg_syd = df_crimes_agg[df_crimes_agg["Postcode"].isin(sydney_post_codes)].copy()
df_crimes_agg_syd.shape


In [ ]:
# Join the postcode data so we can geocode the crime reports using the latlng of each postal area
# The crime data only give postcode level detail but each postcode has diferent neighbouthoods
# As the data is not granular we will allocate the first geocoded neighbourhood in each postcode
# When we come to visualise the data it may be some of the markers are slightly misplaced
df_lookup=df_geo_postcodes_suburb_nsw[["postcode","locality", "lat","long"]].groupby(["postcode"], as_index=False).first()

df_crimes_agg_geo_syd = pd.merge(df_crimes_agg_syd, df_lookup, left_on="Postcode", right_on="postcode", how="left")

# Check that each row received the correct geo encoding
df_crimes_agg_geo_syd.isnull().sum()
df_crimes_agg_geo_syd.shape

In [ ]:
# Create a dataset suitable for visualisation. For this we really want the total reported crimes per postcode
# Group by Postcode, latitude and longitude and sum up each column to the Postcode group
grp = df_crimes_agg_geo_syd.groupby(["Postcode","locality","lat","long"]).sum()

df_crime_vis = grp[["2018"]].copy()

df_crime_vis["min"] = grp.min(axis=1)
df_crime_vis["max"] = grp.max(axis=1)
df_crime_vis["mean"] = grp.mean(axis=1)
df_crime_vis["sum"] = grp.sum(axis=1)
df_crime_vis["rank"] = df_crime_vis["mean"].rank(ascending=0)
df_crime_vis.reset_index(inplace=True)
df_crime_vis

df_crime_vis[df_crime_vis["Postcode"].isin(["2026","2027","2031","2034","2035","2093","2094","2095","2096"])]

In [ ]:
# Create crime map of Sydney using latitude and longitude of the postcodes
from folium.plugins import HeatMap

crime_map_a = folium.Map(location=sydney_lat_lng, control_scale=False, zoom_start=12)

HeatMap(data=df_crime_vis[["lat", "long", "2018"]].values.tolist(), 
        name="crime_heatmap_a",
        min_opacity=0.0,
        max_val=0.75,
        radius=40,
        blur=60,
        max_zoom=16).add_to(crime_map_a)


# Add markers to map for each data point
for lat, lng, postcode, locality, crimes in zip(df_crime_vis["lat"], df_crime_vis["long"], df_crime_vis["Postcode"], df_crime_vis["locality"], df_crime_vis["2018"]):
    popuptext = "<b>{} 2018</b><br>Crimes Reported: {}".format(locality.title(), crimes)
    popuphtml = folium.Html(popuptext, script=True)
    popup = folium.Popup(popuphtml, max_width=250, min_width=250)
    icon = folium.Icon(color="gray", icon="info-sign")
    folium.Marker(location=[lat,lng] , popup = popup, icon=icon).add_to(crime_map_a)
    

# Add markers to map for each data point
#for lat, lng, postcode, locality, crimes in zip(df_crime_vis["lat"], df_crime_vis["long"], df_crime_vis["Postcode"], df_crime_vis["locality"], df_crime_vis["2018"]):
#    label = '{} {} Crimes in 2018: {}'.format(locality, postcode, crimes)
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='white',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(crime_map_a) 

crime_map_a

In [ ]:
# Create crime map of Sydney using latitude and longitude of the postcodes
from folium.plugins import HeatMap

crime_map_b = folium.Map(location=sydney_lat_lng, control_scale=False, tiles='Stamen Toner', zoom_start=12)
HeatMap(data=df_crimes_agg_geo_syd[["lat","long", "2018"]].values.tolist(), 
        name="crime_heatmap_b",
        min_opacity=0.0,
        max_val=0.75,
        radius=40,
        blur=60,
        max_zoom=16).add_to(crime_map_b)

# Add markers to map for each data point
for lat, lng, postcode, locality, crimes in zip(df_crime_vis["lat"], df_crime_vis["long"], df_crime_vis["Postcode"], df_crime_vis["locality"], df_crime_vis["2018"]):
    label = '{} {} Crimes in 2018: {}'.format(locality, postcode, crimes)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='white',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(crime_map_b) 
    
crime_map_b

In [ ]:
# Visualise Crime Data with Choropleth Map
import branca

crime_map_c = folium.Map(location=sydney_lat_lng, control_scale=False, zoom_start=12)
geo_json_file = "./data/au-postcodes-100-percent_geo.json"
bins = list(df_crime_vis["2018"].quantile([0, 0.25, 0.5, 0.75, 1]))
bins = [0, 1000, 5000,10000, 20000, 30000]

folium.Choropleth(
    geo_data=geo_json_file,
    name="choropleth",
    data=df_crime_vis,
    columns=["Postcode", "2018"],
    key_on="properties.POA_CODE",
    fill_color="YlOrRd",
    #fill_color="BuPu",
    #fill_color=color_scale,
    fill_opacity=0.5,
    line_opacity=0.8,
    legend_name="Mean Reported Crimes per year",
    bins=bins,
    reset=True
).add_to(crime_map_c)

folium.LayerControl().add_to(crime_map_c)
crime_map_c


# Load the House Rental Price Data
This is obtained from Rental Lodgements agency, an indepenent organisation that holds tenants security deposits ensuring landlords do not unfairly withold repayment. The data set was obtained from the New South Wales Local Government Fair Trading web site and mey be found [here](https://www.fairtrading.nsw.gov.au/about-fair-trading/data-and-statistics/rental-bond-data)

## Rental Lodgements Dataset for Year 2019
| Field | Description |
| --- | --- |
| Lodgement Date | Date the Deposit was Lodged |
| PostCode | Postal Code of Rental Property |
| Dwelling Type | Code for type of property |
| Bedrooms | Number of Bedrooms |
| Weekly Rent | Weekly Rent in AUD |

### Permissible Values for Dwelling Type
Type of rented premises as provided by the agent of landlord. 
+ (F) Flat or unit
+ (H) House
+ (T) Terrace, townhouse or semi-detached
+ (O) Other
+ (U) Unknown  

Note a Dwelling Type of "Other" includes rented rooms, garages and car spaces.

### Number of Bedrooms
+ A value of 0 indicates garage space or rental car space
+ A value of U indicates Not specified


In [ ]:
# Download the dataset
rental_data_source = "https://www.fairtrading.nsw.gov.au/__data/assets/excel_doc/0008/588662/RentalBond_Lodgements_Year_2019.xlsx"
rental_data_target = "./data/RentalBond_Lodgements_year_2019.xlsx"

!wget -O $rental_data_target $rental_data_source

In [ ]:
# Load the data file from teh first sheet of the spreadsheet
# Note this includes 2 header rows that should be skipped
df_rentals = pd.read_excel(rental_data_target, sheet_name=0, skiprows=2)
df_rentals.head()

In [ ]:
# Decode the Category Data With a Description and Add a New Column
dwelling_dicts = {"F":"Flat", "H":"House", "T":"Townhouse", "U":"Unknown", "O":"Other"}
df_rentals["Dwelling Desc"] = df_rentals["Dwelling Type"].apply(lambda x: dwelling_dicts[x])

# Move the dwelling type columns next to each other
col_orders = df_rentals.columns[0:3].to_list()
col_orders = col_orders + df_rentals.columns[-1:].to_list()
col_orders = col_orders + df_rentals.columns[3:-1].to_list()
df_rentals = df_rentals[col_orders]

# Get the shape of the data
print(df_rentals.shape)

# Check the result
df_rentals.head()

In [ ]:
# Drop Rows where the Rental Price is Unspecified = "U" as these add no useful data 
# We are only interested in the rental price
rows = df_rentals[df_rentals["Weekly Rent"]=="U"].index
df_rentals.drop(rows, axis=0, inplace=True)
print(df_rentals.shape)

# Drop Rows where the Dwelling Type is Unspecified = "U" ot "O" as these add no useful data 
rows = df_rentals[df_rentals["Dwelling Type"].isin(["U","O"])].index
df_rentals.drop(rows, axis=0, inplace=True)
print(df_rentals.shape)

# Drop Rows where the Bedrooms is Unspecified = "U" as these add no useful data 
rows = df_rentals[df_rentals["Bedrooms"]=="U"].index
df_rentals.drop(rows, axis=0, inplace=True)
print(df_rentals.shape)

In [ ]:
df_rentals.head()